# Error mitigation

This notebook contains examples for using our Error Mitigation module with Qiskit.

## Theoretical background

In this section description of proposed method will be provided. There are several important things to note first. Our method is targeted at such error model wherein dominant noise is classical. In practice, alongiside classical noise, there also exists coherent noise. Information about both of these noises can be, in a way, extracted from QDT. The usage of QDT module has been extracted in previous section.

Let $M$ denote POVM describing noisy detector and $P$ denote ideal measurement. Then classical error can be mathematically expressed as a stochastic, invertable map $\Lambda$ in such way, that:

$$ M = \Lambda P.$$

If that's that case, then following correction procedure can be applied. Imagine, we've access to experimental statistics $p_{exp}$ and we want to correct them in such way, that would allow us to obtain ideal one $p_{ideal}$. Due to the assumed nature of the noise, we can write, that:

$$ p_{exp} = \Lambda p_{ideal}. $$

We've assumed, when describing the noise, that $\Lambda$ is invertible. If so, then by multiplying last equation by $\Lambda^{-1}$ from both sides, we obtain

$$ \Lambda^{-1} p_{exp} = p_{ideal}. $$

Effectively, by this kind of postprocessing, we obtain statistics close to what we would normally obtain on non-noisy devices.

## Mitigating error using our module

Out error mitigation approach is based on QDT. Therefore in order to use it, we first need to obtain POVM describing our detectors. In order to do that, we can follow the steps from our [QDT tutorial](https://github.com/fbm2718/QREM/blob/master/QDT_Tutorial.ipynb). We do just that.


In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter


# Choose qubit indices
QDT_qubit_index = [3]

# Select probe kets
QDT_probe_kets = povmtools.pauli_probe_eigenkets

# Generate circuits
QDT_circuits = detector_tomography_circuits(QDT_qubit_index, QDT_probe_kets)

# Get QDT circuits results
backend = Aer.get_backend('qasm_simulator')  #  Get backed
shots_number = 2000  # Define number of measurement repetitions
QDT_job = execute(QDT_circuits, backend=backend, shots=shots_number)
results = QDT_job.result()

# Get ml_povm_estimator using DTF and results
DTF = DetectorTomographyFitter()
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator([results], QDT_probe_kets)

Now, that we have POVM, we can create QDTErrorMitigator object and prepare it.

In [ ]:
# Creation and preparation of QDTErrorMitigator
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With prepared mitigator object we gain access to several useful functionalities. For example, we can:
* Correct results of qiskit job by using apply_correction_to_qiskit_job(Results) method.
* Access transition and correction matrices obtained from POVM given during preparation.

In order to properly analyse the results of correction procedure, one have to be aware that in some cases raw application of $\Lambda^{-1}$ to the results may yield quasiprobability (instead of probability) vectors. In such scenario our method calculates closest probability vectors and returns them instead. Distances from raw quasiprobabilities to returned probabilities, can be accessed via distances_from_closest_probability_vector member of mitigator object.

### Error bounds

With access to POVM and the correction and transition matrices, we are able to calculate bounds on several errors. In particular, using povtools module, we can calculate:
*statistical error bound (using get_statistical_erro, coherent error bound and correction error bound.  